

```

```

# NEXT WORD PREDICTION MODEL

Name-Rishi Raghuwanshi

Virtual Internship@LetsGrowMore

TASK- NEXT WORD PREDICTION MODEL

**IMPORTING THE LIBRARIES**

In [ ]:
import numpy as np
import tensorflow as tf
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding,Activation,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
path = '1661-0.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [ ]:
words


['project',
 'gutenberg',
 's',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'by',
 'arthur',
 'conan',
 'doyle',
 'this',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 'you',
 'may',
 'copy',
 'it',
 'give',
 'it',
 'away',
 'or',
 're',
 'use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www',
 'gutenberg',
 'net',
 'title',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'author',
 'arthur',
 'conan',
 'doyle',
 'release',
 'date',
 'november',
 '29',
 '2002',
 'ebook',
 '1661',
 'last',
 'updated',
 'may',
 '20',
 '2019',
 'language',
 'english',
 'character',
 'set',
 'encoding',
 'utf',
 '8',
 'start',
 'of',
 'this',
 'project',
 'gutenberg',
 'ebook',
 'the',
 'adventures',
 'of',
 'sherlock',
 'holmes',
 'produced',
 'by',
 'an',
 'ano

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
WORD_LENGTH=5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [ ]:
X=np.zeros((len(prev_words),WORD_LENGTH,len(unique_words)),dtype=bool)
y=np.zeros((len(next_words),len(unique_words)),dtype=bool)

In [ ]:
print(X[0][0])

[False False False ... False False False]


In [ ]:
model=Sequential()
model.add(LSTM(120,input_shape=(WORD_LENGTH,len(unique_words))))
model.add(Dense(len(unique_words),activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
optimizer=RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history=model.fit(X,y,validation_split=0.05,batch_size=128,epochs=2,shuffle=True).history

Epoch 1/2
811/811 ━━━━━━━━━━━━━━━━━━━━ 248s 303ms/step - accuracy: 0.0090 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/2
811/811 ━━━━━━━━━━━━━━━━━━━━ 265s 308ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


In [ ]:
model.save('keras_next_word_model.h5')
pickle.dump(history,open('history.p','wb'))
model=load_model('keras_next_word_model.h5')
history=pickle.load(open('history.p','rb'))

Testing model

In [ ]:
def prepare_input(text):
  x=np.zeros((1,WORD_LENGTH,len(unique_words)))
  for t,word in enumerate(text.split()):
        print(word)
        x[0,t,unique_word_index[word]]=1
  return x
prepare_input("how are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [ ]:
def sample(preds,top_n=3):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  return heapq.nlargest(top_n,range(len(preds)),preds.take)

In [ ]:
def predict_completion(text,n=3):
  if text=='':
    return('0')
  x=prepare_input(text)
  preds=model.predict(x,verbose=0)[0]
  next_indices=sample(preds,n)
  return [unique_words[idx] for idx in next_indices]

In [ ]:
q='how are you'
print("correct sentence: ",q)
seq=' '.join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ",predict_completion(seq))

correct sentence:  how are you
Sequence:  how are you
how
are
you
next possible words:  ['factor', 'carbuncle', '1500']


# **THANK YOU!!**